In [113]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [114]:
endpoint = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

res = requests.get(endpoint)

soup = BeautifulSoup(res.text)



In [115]:
us_state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

In [116]:
# def get_all():
#     districts = ['01', '02', ...] 
#     for state in us_state_abbreviations:
#         endpoint = f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state}{districts}&spec=N'

#         res = requests.get(endpoint)

#         soup = BeautifulSoup(res.text)

#         soup.find()

In [117]:
# candidate_elements = soup.find_all('h2')

# # Placeholder for our extracted data

# for element in candidate_elements:
#     text = element.get_text(strip=True)
#     print(text)

In [160]:
# Your existing code
endpoint = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'
res = requests.get(endpoint)
soup = BeautifulSoup(res.text, 'html.parser')
pattern_candidate = re.compile(r"(.+?) \((R|D|I)\)( •Incumbent•Winner)?(\(([\d.]+)% of vote\))?")
# Your existing code...
data = []

# Extract candidate data
for element in soup.find_all('h2'):
    text = element.get_text(strip=True)
    match = pattern_candidate.match(text)
    if match:
        name, party, incumbent_winner, _, vote_percentage = match.groups()
        incumbent = incumbent_winner is not None
        winner = incumbent  # Assumes if they are incumbent they are also the winner
        data.append({
            "Name": name,
            "Party": party,
            "Incumbent": incumbent,
            "Winner": winner,
            "Vote Percentage": vote_percentage,
            "Raised": None,
            "Spent": None
        })

financial_tables = soup.select('table', _class="Members--table")
print(financial_tables)
# Extract 'Raised' and 'Spent' data
for i, table in enumerate(financial_tables):
    if i < len(data):  # Ensure we have a corresponding candidate
        financial_dict = {}
        rows = table.find_all('tr')
        for row in rows:
            label = row.find('td')
            if label:
                label = label.get_text(strip=True)
                print(label)
            value_element = row.find('td', class_='Members--number')
            print(f"value element: {value_element}")
            # Check whether the element was found before accessing its text
            if value_element is not None:
                value = value_element.get_text(strip=True).replace("$", "").replace(",", "")
            else:
                value = None
            
            # Assign values to financial_dict
            if label in ["Raised:", "Spent:"]:
                financial_dict[label.rstrip(':')] = value
        
        # Add financial data to respective candidate
        if financial_dict:
            data[i]["Raised"] = financial_dict.get('Raised')
            data[i]["Spent"] = financial_dict.get('Spent')

# Creating DataFrame
df = pd.DataFrame(data)
print(df)


[<table border="0" cellpadding="1" cellspacing="0" class="Members--table">
<tr>
<td>Raised:</td>
<td class="Members--number">$1,194,960</td>
<td width="33%">
<div class="Members--bar-wrapper">
<div class="Members--bar Members--bar-indivs" style="width:100.0%;"></div>
</div>
</td>
</tr>
<tr>
<td>Spent:</td>
<td class="Members--number">$935,487</td>
<td width="33%">
<div class="Members--bar-wrapper">
<div class="Members--bar Members--bar-indivs" style="width:78.28599300862228%;"></div>
</div>
</td>
</tr>
<tr>
<td>Cash on Hand:</td>
<td class="Members--number">$287,889</td>
<td width="33%">
<div class="Members--bar-wrapper">
<div class="Members--bar Members--bar-indivs" style="width:24.091888997800908%;"></div>
</div>
</td>
</tr>
</table>, <table>
<thead>
<tr>
<th scope="col">Type of Contribution</th>
<th class="number" scope="col">Amount</th>
<th class="number" scope="col">Percentage</th>
</tr>
</thead>
<tbody>
<tr>
<td>Small Individual Contributions (≤ $200)</td>
<td class="number">$0</